In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures 
from sklearn.linear_model import LinearRegression,LogisticRegression, Lasso, Ridge, BayesianRidge,ElasticNet
import patsy as ps

In [2]:
df_train = pd.read_csv('TRAIN.csv')
df_test = pd.read_csv('TEST.csv')
df_train

,LID,MRP,CHANNEL,SELLER,ORATE,CRATE,OUPB,OLTERM,ODATE,FPD,...,OCCSTAT,STATE,MSA,ZIP,MIP,CLDS,MIT,RMI,FORECLOSED,NMONTHS
0,8.118470e+11,3/1/2000,C,Suntrust Mortgage Inc.,8.375,8.375,58000,360,7/1/1999,9/1/1999,...,P,OK,36420,730,NaN,0,NaN,N,False,12
1,9.288980e+11,2/1/2000,C,Amtrust Bank,7.500,7.500,204000,180,12/1/1999,2/1/2000,...,P,WA,48300,988,NaN,0,NaN,N,False,21
2,2.229740e+11,1/1/2000,C,"Jpmorgan Chase Bank, Na",7.750,7.750,181000,360,12/1/1999,2/1/2000,...,P,VA,40060,238,30.0,0,1.0,N,False,54
3,5.939500e+11,2/1/2000,B,"Bank Of America, N.A.",8.625,8.625,135000,360,1/1/2000,3/1/2000,...,P,AZ,38060,852,30.0,0,2.0,N,False,16
4,6.773390e+11,2/1/2000,C,Other,7.750,7.750,189000,360,12/1/1999,2/1/2000,...,P,IN,29140,479,NaN,0,NaN,N,False,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164570,8.194500e+11,2/1/2000,C,Amtrust Bank,7.875,7.875,106000,180,11/1/1999,1/1/2000,...,I,MI,11460,481,NaN,0,NaN,N,False,24
164571,6.558220e+11,3/1/2000,R,"Jpmorgan Chase Bank, Na",8.000,8.000,144000,180,2/1/2000,4/1/2000,...,P,CA,41740,920,NaN,0,NaN,N,False,19
164572,6.577330e+11,3/1/2000,R,First Tennessee Bank National Association,8.250,8.250,88000,360,2/1/2000,3/1/2000,...,P,WV,26580,255,30.0,0,1.0,N,False,12
164573,3.145290e+11,1/1/2000,R,"Ge Mortgage Services, Llc",7.875,7.875,99000,240,10/1/1999,12/1/1999,...,P,NY,35620,115,NaN,0,NaN,N,False,217


In [3]:
df_variable = pd.read_csv('VariablesUsed.csv')
df_variable.groupby(['used'])['used'].count()

used
False    76
True     32
Name: used, dtype: int64

In [4]:
df_train = df_train.drop(['CLTV','MIP','MIT','CBCSAO'],axis = 1)
df_test = df_test.drop(['CLTV','MIP','MIT','CBCSAO'],axis = 1)
df_train

,LID,MRP,CHANNEL,SELLER,ORATE,CRATE,OUPB,OLTERM,ODATE,FPD,...,PTYPE,NUNITS,OCCSTAT,STATE,MSA,ZIP,CLDS,RMI,FORECLOSED,NMONTHS
0,8.118470e+11,3/1/2000,C,Suntrust Mortgage Inc.,8.375,8.375,58000,360,7/1/1999,9/1/1999,...,SF,1,P,OK,36420,730,0,N,False,12
1,9.288980e+11,2/1/2000,C,Amtrust Bank,7.500,7.500,204000,180,12/1/1999,2/1/2000,...,SF,1,P,WA,48300,988,0,N,False,21
2,2.229740e+11,1/1/2000,C,"Jpmorgan Chase Bank, Na",7.750,7.750,181000,360,12/1/1999,2/1/2000,...,SF,1,P,VA,40060,238,0,N,False,54
3,5.939500e+11,2/1/2000,B,"Bank Of America, N.A.",8.625,8.625,135000,360,1/1/2000,3/1/2000,...,PU,1,P,AZ,38060,852,0,N,False,16
4,6.773390e+11,2/1/2000,C,Other,7.750,7.750,189000,360,12/1/1999,2/1/2000,...,SF,1,P,IN,29140,479,0,N,False,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164570,8.194500e+11,2/1/2000,C,Amtrust Bank,7.875,7.875,106000,180,11/1/1999,1/1/2000,...,SF,1,I,MI,11460,481,0,N,False,24
164571,6.558220e+11,3/1/2000,R,"Jpmorgan Chase Bank, Na",8.000,8.000,144000,180,2/1/2000,4/1/2000,...,SF,1,P,CA,41740,920,0,N,False,19
164572,6.577330e+11,3/1/2000,R,First Tennessee Bank National Association,8.250,8.250,88000,360,2/1/2000,3/1/2000,...,SF,1,P,WV,26580,255,0,N,False,12
164573,3.145290e+11,1/1/2000,R,"Ge Mortgage Services, Llc",7.875,7.875,99000,240,10/1/1999,12/1/1999,...,SF,1,P,NY,35620,115,0,N,False,217


In [42]:
import datetime
df_train['MRP'].datetime.toordinal

AttributeError: 'Series' object has no attribute 'datetime'

In [5]:
df_train.isnull().sum()

LID              0
MRP              0
CHANNEL          0
SELLER           0
ORATE            0
CRATE            0
OUPB             0
OLTERM           0
ODATE            0
FPD              0
LA               0
TMTLM            0
RMTM             0
MDATE            0
LOTV             1
NB             198
DTI           6998
BCSAO         3981
FTHBI          218
LP               0
PTYPE            0
NUNITS           0
OCCSTAT          0
STATE            0
MSA              0
ZIP              0
CLDS             0
RMI              0
FORECLOSED       0
NMONTHS          0
dtype: int64

In [6]:
#Date = ['MRP','ODATE','FPD']
#for i in Date:
#    X_train[i]= pd.to_datetime(X_train[i])
#    X_test[i]= pd.to_datetime(X_test[i])
#    X_train[i] =(X_train[i]-min(X_train[i])).dt.days
#    X_test[i] =(X_test[i]-min(X_test[i])).dt.days

In [7]:
#ss = StandardScaler()
#ss.fit(X_train)
#X_train = ss.transform(X_train)
#X_test = ss.transform(X_test)

In [8]:
df_train["LOTV"] = df_train["LOTV"].fillna(df_train["LOTV"].median)
df_train["NB"] = df_train["NB"].fillna(df_train["NB"].median)
df_train["DTI"] = df_train["DTI"].fillna(df_train["DTI"].median)
df_train["BCSAO"] = df_train["BCSAO"].fillna(df_train["BCSAO"].median)
df_train["FTHBI"] = df_train["FTHBI"].fillna(df_train["FTHBI"].median)

In [9]:
df_train.isnull().sum()

LID           0
MRP           0
CHANNEL       0
SELLER        0
ORATE         0
CRATE         0
OUPB          0
OLTERM        0
ODATE         0
FPD           0
LA            0
TMTLM         0
RMTM          0
MDATE         0
LOTV          0
NB            0
DTI           0
BCSAO         0
FTHBI         0
LP            0
PTYPE         0
NUNITS        0
OCCSTAT       0
STATE         0
MSA           0
ZIP           0
CLDS          0
RMI           0
FORECLOSED    0
NMONTHS       0
dtype: int64

In [10]:
X_train = df_train.iloc[:,1:-2]
X_test = df_test.iloc[:,1:]
Y_train = df_train['NMONTHS']
Y_train = Y_train.to_numpy()
names_train = X_train.columns.values
names_test = X_test.columns.values

In [11]:
categorical_cols = ['MRP','CHANNEL','LP','PTYPE','OCCSTAT',"CLDS"]

In [12]:
for col in df_train.columns:
    if col in categorical_cols:
        a={}
        for i in range(len(df_train[col].unique())):
            a[df_train[col].unique()[i]]=i
        work=df_train[col].map(a)
        df_train[col]=work
#encoded = list(X_train.columns)

for col in df_test.columns:
    if col in categorical_cols:
        a={}
        for i in range(len(df_test[col].unique())):
            a[df_test[col].unique()[i]]=i
        work=df_test[col].map(a)
        df_test[col]=work
#encoded = list(X_test.columns)

# Data Prepocessing

In [13]:
len(np.transpose(X_test))

27

In [14]:
#NMONTH_train = pd.DataFrame(Y_train, columns = ['NMONTHS'] )
#df_train['NMONTHS']=NMONTH_train.NMONTHS
#NMONTH_train = NMONTH_train.iloc[:,-1]

In [15]:
N=df_train.shape[0]
perm=np.random.permutation(range(N))
df_train=df_train.loc[perm]
Ntrain=int(.8*N)
Ntest=N-Ntrain
data_train=df_train.iloc[0:Ntrain]
data_test=df_train.iloc[Ntrain:N]
#data_train.astype('object').dtypes
#data_test.astype('object').dtypes

In [16]:
df_test.shape

(82288, 28)

In [17]:
formula="NMONTHS~MRP+CHANNEL+SELLER+ORATE+CRATE+OUPB+OLTERM+ODATE+FPD+LA+TMTLM+RMTM+MDATE+LOTV+NB+DTI+BCSAO+FTHBI+LP+PTYPE"
Y_train,X_train=ps.dmatrices(formula,data_train)
Y_test,X_test=ps.dmatrices(formula,data_test)
Ntest=Y_test.shape[0]
reg= LinearRegression().fit(X_train,Y_train)

Ytest_pred=reg.predict(X_test)
Ytest_pred = np.rint(Ytest_pred)
e=pd.Series((Y_test-Ytest_pred).reshape((Ntest,)))
MAD=e.apply(abs).mean()
print(MAD)


ValueError: X has 499 features, but LinearRegression is expecting 549 features as input.

In [44]:
df_train.groupby(["FPD"])["FPD"].count()

FPD
0       582
1     56182
2     41353
3     24616
4     31341
5      1333
6      4041
7       173
8      2858
9       283
10      442
11      158
12      901
13      192
14        4
15      116
Name: FPD, dtype: int64

binary: RMI
3,4: MRP,CHANNEL,LP,OCCSTAT,PTYPE,CLDS
20: SELLER,STATE
DATE:MRP,ODATE(14),FPD
numerical:ORATE,CRATE,OUPB,OLTERM,LA,TMTLM,RMTM,MDATE,NUNITS,MSA,ZIP,CLDS
    
LOTV,NB,DTI,DCSAO,CBCSAO,FTHBI

In [47]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold 

k = 5
kf = KFold(n_splits=k, random_state=None)
model = LinearRegression()
 
acc_score = []
 
for train_index , test_index in kf.split(df_train):
    X_train_train , X_train_test = df_train.iloc[train_index,:],df_train.iloc[test_index,:]
    Y_train , Y_test = NMONTH_train[train_index] , NMONTH_train[test_index]
     
    model.fit(X_train_train,Y_train)
    pred_values = model.predict(X_train_test)
     
    acc = accuracy_score(pred_values , Y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

NameError: name 'NMONTH_train' is not defined